**Amritesh Pandey**

22-14-02

M.tech Modelling and Simulation

SoCEMS

# Assignment 1(ML Lab)


In [1]:
import pandas as pd
import numpy as np
import copy, math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# Load the data
data = pd.read_csv('Housepriceprediction.csv')
Y = data.SalePrice
X = data.drop(["SalePrice","Id"],axis=1)

**----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**
# Normalizing Data(Scaling)
### z-score normalization 
After z-score normalization, all features will have a mean of 0 and a standard deviation of 1.

To implement z-score normalization, adjust your input values as shown in this formula:
$$x^{(i)}_j = \dfrac{x^{(i)}_j - \mu_j}{\sigma_j} $$ 
where $j$ selects a feature or a column in the $\mathbf{X}$ matrix. $µ_j$ is the mean of all the values for feature (j) and $\sigma_j$ is the standard deviation of feature (j).
$$
\begin{align}
\mu_j &= \frac{1}{m} \sum_{i=0}^{m-1} x^{(i)}_j \\
\sigma^2_j &= \frac{1}{m} \sum_{i=0}^{m-1} (x^{(i)}_j - \mu_j)^2  
\end{align}
$$
**Note - We are normalizing data to converge faster to minimum as large data will take much time to converge.**

In [3]:
def zscore_normalize_features(X):
   
    for i in range(len(X.T)):
        # find the mean of each column/feature
        mu     = np.mean(X, axis=0)                 # mu will have shape (n,)
        # find the standard deviation of each column/feature
        sigma  = np.std(X, axis=0)                  # sigma will have shape (n,)
        # element-wise, subtract mu for that column from each example, divide by std for that column
        X_norm = (X - mu) / sigma      

    return X_norm,mu,sigma
        
   

# Manual Gradient descent Implementation 

###  Model Prediction With Multiple Variables
The model's prediction with multiple variables is given by the linear model:

$$ f_{\mathbf{w},b}(\mathbf{x}) =  w_0x_0 + w_1x_1 +... + w_{n-1}x_{n-1} + b $$
or in vector notation:
$$ f_{\mathbf{w},b}(\mathbf{x}) = \mathbf{w} \cdot \mathbf{x} + b   $$ 
where $\cdot$ is a vector `dot product`

###### We will use this model for linear regreesion here

### compute_cost 

* Iterate over the training examples, and for each example, compute:
    * The prediction of the model for that example 
    $$
    f_{wb}(x^{(i)}) =  wx^{(i)} + b 
    $$
   
    * The cost for that example  $$cost^{(i)} =  (f_{wb} - y^{(i)})^2$$
    

* Return the total cost over all examples
$$J(\mathbf{w},b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} cost^{(i)}$$
  * Here, $m$ is the number of training examples and $\sum$ is the summation operator

In [4]:
def compute_cost(X, y, w, b): 
    
    m = X.shape[0]
    cost = 0.0
    for i in range(m):                                
        f_wb_i = np.dot(X[i], w) + b           
        cost = cost + (f_wb_i - y[i])**2
    total_cost = cost / (2 * m)                         
    return total_cost

## compute gradient

$$\begin{align*} \text{repeat}&\text{ until convergence:} \; \lbrace \newline
\;  w &= w -  \alpha \frac{\partial J(w,b)}{\partial w}   \; \newline 
 b &= b -  \alpha \frac{\partial J(w,b)}{\partial b}  \newline \rbrace
\end{align*}$$
where, parameters $w$, $b$ are updated simultaneously.  
The gradient is defined as:
$$
\begin{align}
\frac{\partial J(w,b)}{\partial w}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})x^{(i)} \\
  \frac{\partial J(w,b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)}) \\
\end{align}
$$


In [5]:
def compute_gradient(X, y, w, b): 
   
    m,n = X.shape           #(number of examples, number of features)
    dj_dw = np.zeros((n,))
    dj_db = 0.

    for i in range(m):                             
        err = (np.dot(X[i], w) + b) - y[i]   
        for j in range(n):                         
            dj_dw[j] = dj_dw[j] + err * X[i, j]    
        dj_db = dj_db + err                        
    dj_dw = dj_dw / m                                
    dj_db = dj_db / m                                
        
    return dj_db, dj_dw

## Implementing Gradient Descent

In [6]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters): 
   
    
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    
    w = copy.deepcopy(w_in)  #avoid modifying global w within function
    b = b_in
    
    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_db,dj_dw = gradient_function(X, y, w, b)   ##None

        # Update Parameters using w, b, alpha and gradient
        w = w - alpha * dj_dw               ##None
        b = b - alpha * dj_db               ##None
      
        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            J_history.append( cost_function(X, y, w, b))
        
    return w, b, J_history #return final w,b and J history for graphing

In [7]:

y = Y.to_numpy()

x = X.to_numpy()



In [8]:
x,mu_x,sigma_x = zscore_normalize_features(x)

In [9]:
# Divide data into training and validation subsets
x_train, x_valid, y_train, y_valid = train_test_split(x, y, train_size=0.7, test_size=0.3, random_state=0)

In [10]:
# initialize parameters
initial_w = np.zeros(7, dtype = float)
initial_b = 0.
# some gradient descent settings
iterations = 1000
alpha = 50.0e-2
# run gradient descent 
w_final, b_final, J_hist = gradient_descent(x_train, y_train, initial_w, initial_b, compute_cost, compute_gradient, alpha, iterations)
print(f"b,w found by gradient descent: {b_final:0.2f},{w_final} ")

b,w found by gradient descent: 181034.79,[ 6534.88416799 35159.67809733 12463.52872652 24411.35572016
 -2828.40213305  3716.1626345  10749.40638586] 


**---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**
# Validation
Validating w.r.t test data(x_valid,y_valid)

In [11]:
m,_ = x_valid.shape
prediction = [np.dot(x_valid[i], w_final) + b_final for i in range(m)]
     

### gradient descent r2score

In [12]:
grad_r2 = r2_score(y_valid,prediction)*100

**------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**
# SKLearn Model

In [13]:
model = LinearRegression()
model.fit(x_train,y_train)

LinearRegression()

In [14]:
y_pred = model.predict(x_valid)

### sklearn r2score

In [15]:
sk_r2 = r2_score(y_valid,y_pred)*100

**-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**
# Normal Eqn Linear Regression

$y=X\beta+\varepsilon$

$\hat{\beta}=(X^TX)^{-1}X^Ty$

In [16]:
m = x_train.shape[0]
x_train = np.insert(x_train , [0], [1], axis=1)
y_train = y_train.reshape(m,1)
theta = np.dot(np.linalg.inv(np.dot(x_train.T, x_train)), np.dot(x_train.T, y_train))

In [17]:
m = x_valid.shape[0]
x_valid = np.insert(x_valid , [0], np.ones((m,1)), axis=1)
preds = np.dot(x_valid, theta)

In [18]:
norm_r2 = r2_score(y_valid,preds)*100

## Comparing Model 

In [19]:
model_comparison = {'Normal Equation':norm_r2,'Gradient Descent': grad_r2, 'sklearn':sk_r2}

In [21]:
pd.DataFrame(model_comparison,index=['R2 Value (%)']).T

,R2 Value (%)
Normal Equation,70.659006
Gradient Descent,70.659006
sklearn,70.659006
